In [3]:
from openai import OpenAI
from qdrant_client import QdrantClient
from dotenv import load_dotenv
import json
import requests
from os import environ

load_dotenv()

True

In [14]:
QDRANT_URL = environ.get('QDRANT_CLOUD_URL')
QDRANT_API_KEY = environ.get('QDRANT_API_KEY')
COLLECTION_NAME = 'lotr-characters'
EMBEDDING_DIMENSION = 512
JINA_EMBEDDING_MODEL = "jina-embeddings-v4"
JINA_URL = "https://api.jina.ai/v1/embeddings"
JINA_API_KEY = environ.get('JINA_API_KEY')
QUERYING_TASK = "retrieval.query"
OPENAI_MODEL = "gpt-4o-mini"
OPENAI_TEMPERATURE = 0.5

In [6]:
openai_client = OpenAI()
qd_client = QdrantClient(url=QDRANT_URL, api_key=QDRANT_API_KEY)

In [7]:
def create_jina_embedding(input_text: str)-> list:
    """
    Create embedding using Jina API
    Returns a single embedding vector (list of floats)
    """
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {JINA_API_KEY}",
    }
    data = {
        "input": [input_text],
        "model": JINA_EMBEDDING_MODEL,
        "dimensions": EMBEDDING_DIMENSION,
        "task": QUERYING_TASK,
        "late_chunking": True,
    }
    try:
        res = requests.post(url=JINA_URL, headers=headers, json=data, timeout=30)
        if res.status_code == 200:
            embedding = res.json()["data"][0]["embedding"]
            return embedding
        else:
            raise Exception(f"Jina API error: {res.status_code} - {res.text}")
    except requests.RequestException as e:
        raise Exception(f"Request failed: {str(e)}")

In [8]:
def search(query: str, limit: int = 5):
    """
    Updated search function to use Jina API for query embedding
    """
    try:
        # Create embedding for the search query using Jina API
        query_embedding = create_jina_embedding(input_text=query)
        
        query_points = qd_client.query_points(
            collection_name=COLLECTION_NAME,
            query=query_embedding,
            limit=limit,
            with_payload=True
        )
        results = [point.payload for point in query_points.points]

        return results
    except Exception as e:
        print(f"Error during search: {str(e)}")
        return None

In [24]:
def build_prompt (query: str, search_results: list[dict[str,str]]):
    raw_user_prompt = """
Context from database:
{retrieved_context}

User question:
{user_question}

Answer the question using ONLY the context above.
""".strip()
    
    system_prompt = """
You are a helpful lore expert on J.R.R. Tolkien's Middle-earth. 
You can only answer questions about characters using the provided context retrieved from the database. 
The context includes structured information such as: name, race, titles, realm, family relations, birth and death dates, and short descriptions.

Guidelines:
- If the answer is found in the context, respond clearly and directly.
- If the answer is not in the context, say you don’t know or that the information was not provided.
- Do not invent new facts outside the context.
- Keep your answers concise, but include all relevant details from the context.
- If the user asks for speculation (e.g., "what would happen if X met Y?"), you can summarize based only on what the context says about their traits.
""".strip()
    
    user_prompt = raw_user_prompt.format(retrieved_context=search_results, user_question=query).strip()
    return user_prompt, system_prompt

In [12]:
def format_hits_response(hits: list[dict[str, str|None]]):
    """Format the results into text to plug into chatGPT"""
    character_data = []
    for hit in hits:
        basic_fields = ['race', 'gender', 'realm', 'culture', 'birth', 'death', 'spouse', 'hair', 'height', 'biography', 'history']
        character = {}
        character.update([(field, hit[field]) for field in basic_fields if hit.get(field)])
        character_data.append(character)
    
    return json.dumps(character_data, indent=2)


In [15]:
def llm(user_prompt: str, system_prompt: str):
    """ llm function to call openAI with our specific prompts"""
    res = openai_client.chat.completions.create(
        model=OPENAI_MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=OPENAI_TEMPERATURE
    )
    return res.choices[0].message.content

In [32]:
def answer_lotr(query: str):
    hits = search(query=query, limit=6)
    formatted_hits = format_hits_response(hits=hits)
    user_prompt,system_prompt = build_prompt(query=query, search_results=formatted_hits)
    res = llm(user_prompt=user_prompt, system_prompt=system_prompt)
    return res

In [31]:
answer_lotr(query="who killed the witch king of angmar?")

'The Witch-king of Angmar was killed by Éowyn during the Battle of the Pelennor Fields. She challenged him, and with the help of Merry, she was able to defeat him, fulfilling the prophecy that he would not fall by the hand of man.'